## PyTorch
-    A replacement for numpy to use the power of GPUs
-    a deep learning research platform that provides maximum flexibility and speed


In [ ]:
from __future__ import print_function
import torch

In [ ]:
x = torch.Tensor(5,3) #ゼロ行列
y = torch.rand(5,3) # ランダム行列
# 演算
print(x.add_(y)) #アンダースコアをつける
print(torch.add(x,y))
print(x+y)
# numpy likeに扱える
print(y[:,1],y.mean(), y.std()) 
print(torch.ones(10))
print(torch.ones(10)+1)
# numpyからの変換も可能
import numpy as np
a = np.arange(4).reshape(2,2)
x = torch.from_numpy(a)

## **define-by-run framework**
Chainerの血を受け継ぐPyTorchの大きな特徴。

>  It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

- ネットワークを**動的に**変更できる。
    - 柔軟な設計が可能。
    - 1イテレーションごとに設計を変えられる
    - [参考:ChainerのDefine by Runとは？](http://s0sem0y.hatenablog.com/entry/2017/01/14/060758)

## Autograd
- `torch.autograd`
`Function`と`Variables`で定義すると、**演算結果が保存される**ので、
そのままさかのぼった微分が可能。

下記の式は、

- $x = [[1,2],[3,4]]$
- $y = x^{2} +2$
- $z = y^{2} = (x^{2}+2)^{2}$

$out = \frac{1}{4}\sum_{i} z_{i}z_{i} = (x^{2}+2)^{2}$
より、
$\frac{\partial out}{\partial x_{i}}=x_{i}(x_{i}^{2}+2)$

その結果を返す
 
[more](http://pytorch.org/docs/master/autograd.html)

In [ ]:
from torch.autograd import Variable
import numpy as np
x = Variable(torch.FloatTensor([[1,2],[3,4]]), requires_grad=True)
y = x.float()**2 + 2
print(y.grad_fn)
z = y**2
out = z.mean()
out.backward() # 
print(x.grad)


## NeuralNetの構築

### 学習の手順

- 学習ができるニューラルネットを定義する。
- データセットを繰り返し投げられるようにする。
- インプットに応じて、処理(計算)が走るようにする  
- 誤差(損失)を計算する
- back propagationを実施する
- optimizerによってパラメータを更新する

この中で**データセットを繰り返し投げる以外**の要素を説明していく。
### ライブラリ構成
PyTorchのよく使うパッケージ

|パッケージ|説明|
|---|---|
|`torch`|	NumPyのような配列(テンソル)ライブラリ。GPUも簡単に使える|
|`torch.autograd`|自動微分ライブラリ。Torchで定義されたすべての関数に対して微分を可能にしている|
|`torch.nn`|ネットワークを構成する要素|
|`torch.optim`|パラメータ更新時に使う最適化パッケージ|
|`torch.utils`|DataLoaderなど、その他のユーティリティ関数|


### torch.nn 
ネットワークを構成する要素関連はここに入っている
- `Conv1d`,`Conv2d`,`MaxPool1d`, `AvePool1d`, `AdaptiveAvgPool1d`... : CNN向け
- `RNN`, `LSTM`, `GRU`.... : RNN向け
- `BatchNorm1d`,`BachNorm2d`, `InstanceNorm1d`,... : バッチ正規化
- `functional.hoge` : 活性化関数(SoftMax, ReLUなど)  
- その他、dropout, sparse_embeddings,... 

詳しくは[pytorch:nn](http://pytorch.org/docs/master/nn.html)


すべてPyTorch内の関数で渡してやることができれば、back propagation時の微分演算などがスムーズ

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

## 損失関数
誤差の算出のために、いろいろな関数が用意されている。シンプルなのはMean Squared Error。いろんな誤差を知りたい人は、[機械学習で使う指標総まとめ(教師あり学習編)](http://www.procrasist.com/entry/ml-metrics)へ。

- MSELoss
- CrossEntropyLoss
- NLLLoss(negative log likelihood loss) (1d, 2d)
- Poisson NLL Loss
- KLDivLoss (Kullback-Leibler divergence)
- BCELoss (Binary Cross Entropy)
- ...

より詳しくは[公式](http://pytorch.org/docs/master/nn.html)


In [ ]:
output = net(input)
target = Variable(torch.arange(1, 11))  # a dummy target, for example
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

ここまでで、こうなっている
>input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
     
この損失からさかのぼっていく

In [ ]:
net.zero_grad() #とりあえず0で初期化
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)


## 重みのアップデート
学習には、`torch.optim`を使う

### アルゴリズム
- SGD, ASGD, Nesterov-SGD
- Adadelta, Adagrad,  Adam, SparseAdam, Adamax
- LBFGS
- RMSProp, Rprop

### 学習の仕方
`torch.optim.lr_scheduler`で学習率の調整もできる。


In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update


## MNIST
### ネットワークの作成

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

model = Net()
print(model)

## データのロード
- `torchvision`からデータをダウンロード(画像系)
- `torch.utils.data.DataLoader`にて、データの操作

In [ ]:
from torchvision import datasets, transforms
import torch.utils.data
batch_size = 50
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=1000)

### 実行

In [ ]:
# 損失関数
optimizer = optim.SGD(model.parameters(), lr=0.02)
model.train()
train_loss = []
train_accu = []
i = 0
for epoch in range(10):
    for data, target in train_loader:
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()    # calc gradients
        train_loss.append(loss.data[0])
        optimizer.step()   # update gradients
        prediction = output.data.max(1)[1]   # first column has actual prob.
        accuracy = prediction.eq(target.data).sum()/batch_size*100
        train_accu.append(accuracy)
        if i % 1000 == 0:
            print('Train Step: {}\tLoss: {:.3f}\tAccuracy: {:.3f}'.format(i, loss.data[0], accuracy))
        i += 1
